# source

> Source service for federated transcription queries via DuckDB

In [ ]:
#| default_exp services.source

In [ ]:
#| export
from typing import List, Dict, Any, Optional, Tuple
from pathlib import Path
import json

from cjm_plugin_system.core.manager import PluginManager
from cjm_plugin_system.core.metadata import PluginMeta

from cjm_source_provider.models import SourceBlock, SourceRecord
from cjm_source_provider.protocols import SourceProvider

## TranscriptionDBProvider

A `SourceProvider` implementation for transcription SQLite databases. Each provider instance wraps a single database file (either from a plugin or an external path).

In [ ]:
#| export
class TranscriptionDBProvider:
    """SourceProvider for transcription SQLite databases."""
    
    # Required columns for transcription schema
    REQUIRED_COLUMNS = frozenset(["job_id", "text", "created_at"])
    
    def __init__(
        self,
        db_path: str,  # Path to SQLite database file
        name: str,  # Display name for this provider
        provider_id: Optional[str] = None  # Unique ID (defaults to db_path)
    ):
        """Initialize provider for a transcription database."""
        self._db_path = Path(db_path)
        self._name = name
        self._id = provider_id or str(db_path)
    
    @property
    def provider_id(self) -> str:  # Unique identifier
        """Unique identifier for this provider instance."""
        return self._id
    
    @property
    def provider_name(self) -> str:  # Display name
        """Human-readable name for display."""
        return self._name
    
    @property
    def provider_type(self) -> str:  # Provider category
        """Provider type category."""
        return "transcription_db"
    
    @property
    def db_path(self) -> Path:  # Database file path
        """Path to the underlying database file."""
        return self._db_path
    
    def is_available(self) -> bool:  # Whether database exists and is accessible
        """Check if the database file exists and is accessible."""
        return self._db_path.exists() and self._db_path.suffix == '.db'
    
    def validate_schema(self) -> Tuple[bool, str]:  # (is_valid, error_message)
        """Check if database has valid transcription schema."""
        if not self.is_available():
            return False, f"Database file not found: {self._db_path}"

        try:
            import duckdb
        except ImportError:
            return False, "duckdb is required for schema validation"

        con = duckdb.connect()
        try:
            con.execute(
                f"ATTACH '{self._db_path}' AS db (TYPE SQLITE, READ_ONLY TRUE)"
            )

            # Check if transcriptions table exists and get its columns
            try:
                result = con.execute("DESCRIBE db.transcriptions").fetchall()
                column_names = {row[0] for row in result}
            except duckdb.CatalogException:
                return False, "Missing 'transcriptions' table"

            # Check required columns exist
            missing = self.REQUIRED_COLUMNS - column_names
            if missing:
                return False, f"Missing required columns: {', '.join(sorted(missing))}"

            return True, ""
        except Exception as e:
            return False, f"Schema validation error: {e}"
        finally:
            con.close()
    
    def query_records(
        self,
        limit: int = 100  # Maximum records to return
    ) -> List[SourceRecord]:  # List of source records
        """Query transcription records from the database."""
        if not self.is_available():
            return []
        
        try:
            import duckdb
        except ImportError:
            raise ImportError(
                "duckdb is required for database queries. Install with: pip install duckdb"
            )
        
        records: List[SourceRecord] = []
        con = duckdb.connect()
        
        try:
            con.execute(
                f"ATTACH '{self._db_path}' AS db (TYPE SQLITE, READ_ONLY TRUE)"
            )
            
            query = f"""
                SELECT job_id, audio_path, text, metadata, created_at
                FROM db.transcriptions
                ORDER BY created_at DESC
                LIMIT {limit}
            """
            
            rows = con.execute(query).fetchall()
            for row in rows:
                records.append(SourceRecord(
                    record_id=row[0],
                    media_path=row[1] or "",
                    text=row[2] or "",
                    metadata=json.loads(row[3]) if row[3] else {},
                    created_at=row[4] or "",
                    provider_id=self._id
                ))
        except Exception as e:
            print(f"Warning: Could not query {self._name}: {e}")
        finally:
            con.close()
        
        return records
    
    def get_source_block(
        self,
        record_id: str  # Job ID to fetch
    ) -> Optional[SourceBlock]:  # SourceBlock or None if not found
        """Fetch a specific transcription as a SourceBlock."""
        if not self.is_available():
            return None
        
        try:
            import duckdb
        except ImportError:
            raise ImportError("duckdb is required for database queries")
        
        con = duckdb.connect()
        
        try:
            con.execute(
                f"ATTACH '{self._db_path}' AS db (TYPE SQLITE, READ_ONLY TRUE)"
            )
            
            query = """
                SELECT job_id, audio_path, text, metadata
                FROM db.transcriptions
                WHERE job_id = ?
            """
            
            row = con.execute(query, [record_id]).fetchone()
            
            if row:
                metadata = json.loads(row[3]) if row[3] else {}
                return SourceBlock(
                    id=row[0],
                    provider_id=self._id,
                    text=row[2] or "",
                    media_path=row[1],
                    metadata=metadata
                )
            return None
        finally:
            con.close()
    
    @classmethod
    def from_plugin(
        cls,
        meta: PluginMeta  # Plugin metadata with manifest containing db_path
    ) -> Optional["TranscriptionDBProvider"]:  # Provider or None if no valid db_path
        """Create provider from plugin metadata."""
        if not hasattr(meta, 'manifest'):
            return None
        
        db_path = meta.manifest.get('db_path')
        if not db_path or not Path(db_path).exists():
            return None
        
        return cls(
            db_path=db_path,
            name=meta.name,
            provider_id=f"plugin:{meta.name}"
        )
    
    @classmethod
    def from_external_path(
        cls,
        path: str  # Path to external database file
    ) -> Optional["TranscriptionDBProvider"]:  # Provider or None if path invalid
        """Create provider from an external database path."""
        db_path = Path(path)
        if not db_path.exists() or db_path.suffix != '.db':
            return None
        
        return cls(
            db_path=str(db_path),
            name=f"External: {db_path.stem}",
            provider_id=f"external:{path}"
        )

## SourceService

This service provides access to transcription data from multiple sources via DuckDB federation. It queries the SQLite databases of loaded transcription plugins without loading the full plugin workers.

In [ ]:
#| export
class SourceService:
    """Service for federated access to content sources via providers."""
    
    def __init__(
        self,
        plugin_manager: PluginManager,  # Plugin manager for discovering plugin sources
        source_categories: List[str] = None,  # Plugin categories to query (default: ['transcription'])
        external_paths: List[str] = None  # External database paths
    ):
        """Initialize the source service."""
        self._manager = plugin_manager
        self._categories = source_categories or ["transcription"]
        self._providers: Dict[str, SourceProvider] = {}
        
        # Add providers from external paths
        if external_paths:
            for path in external_paths:
                self.add_external_path(path)
    
    # -------------------------------------------------------------------------
    # Provider Management
    # -------------------------------------------------------------------------
    
    def add_provider(
        self,
        provider: SourceProvider  # Provider instance to add
    ) -> bool:  # True if added, False if ID already exists
        """Add a source provider."""
        if provider.provider_id in self._providers:
            return False
        self._providers[provider.provider_id] = provider
        return True
    
    def remove_provider(
        self,
        provider_id: str  # ID of provider to remove
    ) -> bool:  # True if removed, False if not found
        """Remove a source provider by ID."""
        if provider_id in self._providers:
            del self._providers[provider_id]
            return True
        return False
    
    def get_provider(
        self,
        provider_id: str  # ID of provider to get
    ) -> Optional[SourceProvider]:  # Provider or None if not found
        """Get a provider by ID."""
        return self._providers.get(provider_id)
    
    def get_providers(self) -> List[SourceProvider]:  # List of all providers
        """Get all registered providers."""
        return list(self._providers.values())
    
    def get_provider_by_name(
        self,
        name: str  # Provider name to search for
    ) -> Optional[SourceProvider]:  # Provider or None if not found
        """Find a provider by its display name."""
        for provider in self._providers.values():
            if provider.provider_name == name:
                return provider
        return None
    
    def has_provider_for_path(
        self,
        path: str  # Path to check
    ) -> Tuple[bool, Optional[str]]:  # (has_duplicate, existing_provider_name)
        """Check if any provider uses the same resolved database path."""
        # Ensure plugin providers are loaded for complete check
        self.add_plugin_providers()
        
        try:
            resolved = Path(path).resolve()
        except (ValueError, OSError):
            return False, None
        
        for provider in self._providers.values():
            if hasattr(provider, 'db_path'):
                try:
                    provider_resolved = Path(provider.db_path).resolve()
                    if provider_resolved == resolved:
                        return True, provider.provider_name
                except (ValueError, OSError):
                    continue
        
        return False, None
    
    # -------------------------------------------------------------------------
    # Plugin Provider Discovery
    # -------------------------------------------------------------------------
    
    def add_plugin_providers(self) -> int:  # Number of providers added
        """Discover and add providers from loaded plugins."""
        added = 0
        for category in self._categories:
            plugins = self._manager.get_plugins_by_category(category)
            for meta in plugins:
                provider = TranscriptionDBProvider.from_plugin(meta)
                if provider and self.add_provider(provider):
                    added += 1
        return added
    
    # -------------------------------------------------------------------------
    # External Path Management
    # -------------------------------------------------------------------------
    
    def set_external_paths(
        self,
        paths: List[str]  # List of external database paths to set
    ) -> None:
        """Set external database paths (replaces existing external providers)."""
        # Remove existing external providers
        external_ids = [
            pid for pid in self._providers
            if pid.startswith("external:")
        ]
        for pid in external_ids:
            del self._providers[pid]
        
        # Add new paths
        for path in (paths or []):
            self.add_external_path(path)
    
    def add_external_path(
        self,
        path: str  # External database path to add
    ) -> bool:  # True if added, False if already exists or invalid
        """Add an external database as a provider."""
        provider = TranscriptionDBProvider.from_external_path(path)
        if provider:
            return self.add_provider(provider)
        return False
    
    def remove_external_path(
        self,
        path: str  # External database path to remove
    ) -> bool:  # True if removed, False if not found
        """Remove an external database provider."""
        provider_id = f"external:{path}"
        return self.remove_provider(provider_id)
    
    def get_external_paths(self) -> List[str]:  # List of external database paths
        """Get list of external database paths."""
        paths = []
        for pid, provider in self._providers.items():
            if pid.startswith("external:"):
                # Extract path from provider_id
                paths.append(pid[len("external:"):])
        return paths
    
    # -------------------------------------------------------------------------
    # Source Queries
    # -------------------------------------------------------------------------
    
    def get_available_sources(self) -> List[Dict[str, Any]]:  # List of source info dicts
        """Get list of available sources (for UI display)."""
        # First ensure plugin providers are loaded
        self.add_plugin_providers()
        
        sources = []
        for provider in self._providers.values():
            if isinstance(provider, TranscriptionDBProvider) and provider.is_available():
                sources.append({
                    'name': provider.provider_name,
                    'version': 'N/A',
                    'category': 'transcription',
                    'description': str(provider.db_path),
                    'db_path': str(provider.db_path)
                })
        return sources
    
    def query_transcriptions(
        self,
        provider_name: Optional[str] = None,  # Filter by provider name (None for all)
        limit: int = 100  # Maximum number of results per provider
    ) -> List[Dict[str, Any]]:  # List of transcription records
        """Query records from all providers (or a specific one)."""
        # Ensure plugin providers are loaded
        self.add_plugin_providers()
        
        results: List[Dict[str, Any]] = []
        
        for provider in self._providers.values():
            # Filter by name if specified
            if provider_name and provider.provider_name != provider_name:
                continue
            
            # Query records from provider
            records = provider.query_records(limit=limit)
            
            # Convert SourceRecord to dict
            for rec in records:
                results.append(dict(rec))
        
        return results
    
    def get_transcription_by_id(
        self,
        record_id: str,  # Record ID to fetch
        provider_id: str  # Provider ID that owns this record
    ) -> Optional[SourceBlock]:  # SourceBlock or None if not found
        """Get a specific transcription as a SourceBlock."""
        # Ensure plugin providers are loaded
        self.add_plugin_providers()
        
        # Find provider by ID
        provider = self.get_provider(provider_id)
        if not provider:
            return None
        
        return provider.get_source_block(record_id)
    
    def get_source_blocks(
        self,
        selections: List[Dict[str, str]]  # List of {record_id, provider_id} dicts
    ) -> List[SourceBlock]:  # Ordered list of SourceBlocks
        """Fetch multiple records as SourceBlocks in order."""
        blocks = []
        for sel in selections:
            block = self.get_transcription_by_id(
                record_id=sel['record_id'],
                provider_id=sel['provider_id']
            )
            if block:
                blocks.append(block)
        return blocks

## External Database Validation

In [ ]:
#| export
# Default valid extensions for external database files
VALID_DB_EXTENSIONS = ['.db', '.sqlite', '.sqlite3']

In [ ]:
#| export
def validate_and_toggle_external_db(
    source_service: SourceService,  # Source service for duplicate detection
    path: str,  # Path to the .db file
    external_paths: List[str],  # Current external database paths
    valid_extensions: List[str] = None,  # Valid file extensions (default: VALID_DB_EXTENSIONS)
) -> Tuple[List[str], Optional[str]]:  # (updated_paths, error_message or None)
    """Validate and toggle an external database path in the external paths list."""
    if valid_extensions is None:
        valid_extensions = VALID_DB_EXTENSIONS
    
    resolved_path = Path(path).resolve()
    path_str = str(resolved_path)
    
    # Toggle off if already present
    if path_str in external_paths:
        return [p for p in external_paths if p != path_str], None
    
    # Validate file exists and has valid extension
    if not resolved_path.exists() or resolved_path.suffix.lower() not in valid_extensions:
        return list(external_paths), f"Invalid database file: {resolved_path.name}"
    
    # Check for duplicate provider path
    has_dup, existing_name = source_service.has_provider_for_path(path_str)
    if has_dup:
        return list(external_paths), f"Database already loaded as '{existing_name}'"
    
    # Validate database schema
    temp_provider = TranscriptionDBProvider.from_external_path(path_str)
    if not temp_provider:
        return list(external_paths), f"Could not read database: {resolved_path.name}"
    
    is_valid, validation_error = temp_provider.validate_schema()
    if not is_valid:
        return list(external_paths), f"Unsupported schema: {validation_error}"
    
    # All validations passed
    return external_paths + [path_str], None

## Tests

The following cells demonstrate the provider architecture. `TranscriptionDBProvider` can be created from plugin metadata or external paths. `SourceService` manages multiple providers and aggregates queries.

In [ ]:
#| eval: false
# Test TranscriptionDBProvider directly
from pathlib import Path

# Path to existing transcription database
whisper_db = "/mnt/SN850X_8TB_EXT4/Projects/GitHub/cj-mills/cjm-plugin-system/.cjm/data/cjm-transcription-plugin-whisper/whisper_transcriptions.db"

# Create provider from external path
provider = TranscriptionDBProvider.from_external_path(whisper_db)
print(f"Provider: {provider.provider_name}")
print(f"  ID: {provider.provider_id}")
print(f"  Type: {provider.provider_type}")
print(f"  Available: {provider.is_available()}")

Provider: External: whisper_transcriptions
  ID: external:/mnt/SN850X_8TB_EXT4/Projects/GitHub/cj-mills/cjm-plugin-system/.cjm/data/cjm-transcription-plugin-whisper/whisper_transcriptions.db
  Type: transcription_db
  Available: True


In [ ]:
#| eval: false
# Test validate_schema on valid transcription database
is_valid, error_msg = provider.validate_schema()
print(f"Schema validation: {is_valid}")
if not is_valid:
    print(f"  Error: {error_msg}")
else:
    print(f"  Required columns found: {TranscriptionDBProvider.REQUIRED_COLUMNS}")

Schema validation: True
  Required columns found: frozenset({'job_id', 'text', 'created_at'})


In [ ]:
#| eval: false
# Test validate_schema on database with unsupported schema (e.g., VAD database)
vad_db = "/mnt/SN850X_8TB_EXT4/Projects/GitHub/cj-mills/cjm-plugin-system/.cjm/data/cjm-media-plugin-silero-vad/vad_jobs.db"
vad_provider = TranscriptionDBProvider.from_external_path(vad_db)
if vad_provider:
    is_valid, error_msg = vad_provider.validate_schema()
    print(f"VAD database schema validation: {is_valid}")
    print(f"  Error: {error_msg}")

VAD database schema validation: False
  Error: Missing 'transcriptions' table


In [ ]:
#| eval: false
# Test provider query_records
records = provider.query_records(limit=3)
print(f"Found {len(records)} records from provider")

for rec in records:
    print(f"\n  record_id: {rec['record_id']}")
    print(f"  provider_id: {rec['provider_id']}")
    print(f"  text preview: {rec['text'][:60]}...")

Found 3 records from provider

  record_id: job_9065e18d
  provider_id: external:/mnt/SN850X_8TB_EXT4/Projects/GitHub/cj-mills/cjm-plugin-system/.cjm/data/cjm-transcription-plugin-whisper/whisper_transcriptions.db
  text preview: Laying Plans Sun Tzu said, The art of war is of vital import...

  record_id: job_a0b718cc
  provider_id: external:/mnt/SN850X_8TB_EXT4/Projects/GitHub/cj-mills/cjm-plugin-system/.cjm/data/cjm-transcription-plugin-whisper/whisper_transcriptions.db
  text preview: Laying Plans Sun Tzu said, The art of war is of vital import...

  record_id: job_3cbae8d3
  provider_id: external:/mnt/SN850X_8TB_EXT4/Projects/GitHub/cj-mills/cjm-plugin-system/.cjm/data/cjm-transcription-plugin-whisper/whisper_transcriptions.db
  text preview: Laying Plans Sun Tzu said, The art of war is of vital import...


In [ ]:
#| eval: false
# Test provider get_source_block
if records:
    block = provider.get_source_block(records[0]['record_id'])
    if block:
        print(f"Retrieved SourceBlock:")
        print(f"  id: {block.id}")
        print(f"  provider_id: {block.provider_id}")
        print(f"  text length: {len(block.text)} chars")

Retrieved SourceBlock:
  id: job_9065e18d
  provider_id: external:/mnt/SN850X_8TB_EXT4/Projects/GitHub/cj-mills/cjm-plugin-system/.cjm/data/cjm-transcription-plugin-whisper/whisper_transcriptions.db
  text length: 3402 chars


In [ ]:
#| eval: false
# Test SourceService with providers (backward-compatible API)
from cjm_plugin_system.core.manager import PluginManager

# Calculate project root from notebook location
project_root = Path.cwd().parent.parent
manifests_dir = project_root / ".cjm" / "manifests"

# Create plugin manager
manager = PluginManager(search_paths=[manifests_dir])
manager.discover_manifests()
print(f"Discovered {len(manager.discovered)} plugins")

# Initialize service with external path (backward compatible)
source_service = SourceService(
    plugin_manager=manager,
    external_paths=[whisper_db]
)

# Check providers
providers = source_service.get_providers()
print(f"\nRegistered providers: {len(providers)}")
for p in providers:
    print(f"  - {p.provider_name} ({p.provider_type})")

[PluginManager] Discovered manifest: cjm-transcription-plugin-voxtral-hf from /mnt/SN850X_8TB_EXT4/Projects/GitHub/cj-mills/cjm-fasthtml-workflow-transcript-decomp/.cjm/manifests/cjm-transcription-plugin-voxtral-hf.json
[PluginManager] Discovered manifest: cjm-system-monitor-nvidia from /mnt/SN850X_8TB_EXT4/Projects/GitHub/cj-mills/cjm-fasthtml-workflow-transcript-decomp/.cjm/manifests/cjm-system-monitor-nvidia.json
[PluginManager] Discovered manifest: cjm-transcription-plugin-whisper from /mnt/SN850X_8TB_EXT4/Projects/GitHub/cj-mills/cjm-fasthtml-workflow-transcript-decomp/.cjm/manifests/cjm-transcription-plugin-whisper.json
[PluginManager] Discovered manifest: cjm-media-plugin-silero-vad from /mnt/SN850X_8TB_EXT4/Projects/GitHub/cj-mills/cjm-fasthtml-workflow-transcript-decomp/.cjm/manifests/cjm-media-plugin-silero-vad.json
[PluginManager] Discovered manifest: cjm-graph-plugin-sqlite from /mnt/SN850X_8TB_EXT4/Projects/GitHub/cj-mills/cjm-fasthtml-workflow-transcript-decomp/.cjm/manif

Discovered 6 plugins

Registered providers: 1
  - External: whisper_transcriptions (transcription_db)


In [ ]:
#| eval: false
# Test has_provider_for_path for duplicate detection
has_dup, existing_name = source_service.has_provider_for_path(whisper_db)
print(f"Checking if whisper_db is already loaded:")
print(f"  Has duplicate: {has_dup}")
print(f"  Existing provider: {existing_name}")

# Test with a path not yet added
random_path = "/some/random/path.db"
has_dup2, existing_name2 = source_service.has_provider_for_path(random_path)
print(f"\nChecking random path:")
print(f"  Has duplicate: {has_dup2}")

Checking if whisper_db is already loaded:
  Has duplicate: True
  Existing provider: External: whisper_transcriptions

Checking random path:
  Has duplicate: False


In [ ]:
#| eval: false
# Test query_transcriptions (backward-compatible method)
records = source_service.query_transcriptions(limit=3)
print(f"Found {len(records)} transcription records via SourceService")

for rec in records:
    print(f"\n  record_id: {rec['record_id']}")
    print(f"  provider_id: {rec['provider_id']}")
    print(f"  text preview: {rec['text'][:60]}...")

Found 3 transcription records via SourceService

  record_id: job_9065e18d
  provider_id: external:/mnt/SN850X_8TB_EXT4/Projects/GitHub/cj-mills/cjm-plugin-system/.cjm/data/cjm-transcription-plugin-whisper/whisper_transcriptions.db
  text preview: Laying Plans Sun Tzu said, The art of war is of vital import...

  record_id: job_a0b718cc
  provider_id: external:/mnt/SN850X_8TB_EXT4/Projects/GitHub/cj-mills/cjm-plugin-system/.cjm/data/cjm-transcription-plugin-whisper/whisper_transcriptions.db
  text preview: Laying Plans Sun Tzu said, The art of war is of vital import...

  record_id: job_3cbae8d3
  provider_id: external:/mnt/SN850X_8TB_EXT4/Projects/GitHub/cj-mills/cjm-plugin-system/.cjm/data/cjm-transcription-plugin-whisper/whisper_transcriptions.db
  text preview: Laying Plans Sun Tzu said, The art of war is of vital import...


In [ ]:
#| eval: false
# Test get_source_blocks with multiple selections
if len(records) >= 2:
    selections = [
        {'record_id': records[0]['record_id'], 'provider_id': records[0]['provider_id']},
        {'record_id': records[1]['record_id'], 'provider_id': records[1]['provider_id']}
    ]
    
    blocks = source_service.get_source_blocks(selections)
    print(f"Retrieved {len(blocks)} SourceBlocks via SourceService:")
    for i, block in enumerate(blocks):
        print(f"\n  Block {i}:")
        print(f"    id: {block.id}")
        print(f"    text preview: {block.text[:50]}...")

Retrieved 2 SourceBlocks via SourceService:

  Block 0:
    id: job_9065e18d
    text preview: Laying Plans Sun Tzu said, The art of war is of vi...

  Block 1:
    id: job_a0b718cc
    text preview: Laying Plans Sun Tzu said, The art of war is of vi...


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()